<h1>Data Wrangling<h1>
<h2><i>Discovering, Structuring, Cleaning, Enriching, Validating, and Publishing<i><h2>

<h2> What Do You Want to Do? </h2>
<a id = 'Data Loading'># 1. Data Loading</a><br>
<a id = 'Data Inspection'># 2. Data Inspection</a><br>
<a id = 'Data Cleaning'># 3. Data Cleaning</a>

In [1]:
#Structured Data Wrangling
import numpy as np #working with arrays
import pandas as pd #structured data manipulation
import matplotlib.pyplot as plt #to visualize the data
import seaborn as sns #to visualize the data
import scipy.stats #perform statistics
import statsmodels.stats #perform descriptive statistics



#### Tools to Explore
* [OpenRefine](https://openrefine.org/) - cleaning, transforming, & extending
* [Google DataPrep](https://cloud.google.com/dataprep) - visually exploring, cleaning, and preparing structured and unstructured data 
* [Amazon SageMaker DataWrangler](https://aws.amazon.com/sagemaker/data-wrangler/) - prepare tabular and image data for ML
* [Tabula](https://tabula.technology/) - pull tables from PDF




### Common Data Wrangling Steps - GOAL = Prepared, Ideal State for Analysis
* General Understanding
* Missing Data and Dealing with Incomplete Data
* Outliers
* Reshaping or Restructuring
* Centralizing, Merging Data Shources
* Investigate Variables 

# 1. Data Loading
load data from various sources such as CSV files, Excel sheets, databases, etc., into a structured format like Pandas DataFrame

*GOAL: How would you make this efficient? and Handle the Most Common Formats & Configurations*

In [2]:
import json
import sqlite3

def load_data(source, **kwargs):
    """
    Load data from various sources.

    Parameters:
        source (str): The source of the data. Supported values: 'csv', 'excel', 'json', 'database'.
        **kwargs: Additional keyword arguments depending on the data source.

    Returns:
        DataFrame (CSV, Excel, Database) or other appropriate data structure (JSON).

    Example usage:
        # Load CSV data
        csv_data = load_data(source='csv', filepath='data.csv')

        # Load Excel data
        excel_data = load_data(source='excel', filepath='data.xlsx', sheet_name='Sheet1')

        # Load JSON data
        json_data = load_data(source='json', filepath='data.json')

        # Load data from a database
        database_data = load_data(source='database', db_path='example.db', query='SELECT * FROM table_name')
    """
    supported_sources = ['csv', 'excel', 'json', 'database']
    if source not in supported_sources:
        raise ValueError(f"Unsupported data source. Supported sources are: {', '.join(supported_sources)}")

    if source == 'csv':
        filepath = kwargs.pop('filepath')
        if filepath is None:
            raise ValueError("CSV filepath must be provided.")
        return pd.read_csv(filepath, **kwargs)

    elif source == 'excel':
        filepath = kwargs.get('filepath')
        if filepath is None:
            raise ValueError("Excel filepath must be provided.")
        return pd.read_excel(filepath, **kwargs)

    elif source == 'json':
        filepath = kwargs.get('filepath')
        if filepath is None:
            raise ValueError("JSON filepath must be provided.")
        with open(filepath, 'r') as f:
            data = json.load(f)
        return data

    elif source == 'database':
        db_path = kwargs.get('db_path')
        if db_path is None:
            raise ValueError("Database path must be provided.")
        query = kwargs.get('query')
        if query is None:
            raise ValueError("Query must be provided to extract data from the database.")
        conn = sqlite3.connect(db_path)
        data = pd.read_sql(query, conn, **kwargs)
        conn.close()
        return data


In [5]:
df = load_data(source='csv', filepath='/Users/003/Desktop/CalTech AI/4 - Machine Learning/Datasets - Machine Learning/Lesson_04_Regression_and_Its_Applications/4.11_Data_Preparation_Model_Building_and_Performance_Evaluation/housing.csv')

## Deep Copy vs. Shallow Copy

![Copy](https://ibaibhavsingh.wordpress.com/wp-content/uploads/2016/09/screen-shot-2016-09-03-at-4-06-34-pm.png)

<img src = 'Shallow vs Deep.png'>

https://www.geeksforgeeks.org/difference-between-shallow-and-deep-copy-of-a-class/

In [ ]:
from copy import deepcopy
df_copy = deepcopy(df) #Deepcopy of X

# 2. Data Inspection

### Basic Descriptive Statistics

In [10]:
def inspect_data(df):
    """
    Perform a comprehensive inspection of the loaded data.

    Parameters:
        df (DataFrame): The input DataFrame to be inspected.

    Returns:
        None

    Example usage:
        # Load data into a DataFrame (replace 'data.csv' with your data file)
        df = pd.read_csv('data.csv')

        # Perform a comprehensive inspection of the loaded data
        inspect_data(df)
    """
    print("Data Inspection Report:")
    print("-----------------------")
    
    # 1. Basic Information
    print("\n1. Basic Information:")
    display(df.head())
    display(df.tail())
    print(df.info())

    # 2. Summary Statistics
    print("\n2. Summary Statistics:")
    numerical_df = df.select_dtypes(include=np.number)
    summary_stats = numerical_df.describe(include='all').T
    mode_values = numerical_df.mode().T
    median_values = numerical_df.median().T
    variance_values = numerical_df.var().T
    summary_stats['mode'] = mode_values[0]
    summary_stats['median'] = median_values
    summary_stats['variance'] = variance_values
    display(summary_stats)
    print("\n2. Categorical Summary Statistics:")
    display(df.describe(include=['O']))
    
    # 3. Missing Values
    print("\n3. Missing Values:")
    missing_values = df.isnull().sum()
    if missing_values.sum() == 0:
        print("No missing values found.")
    else:
        display(missing_values)
    
    # 4. Duplicate Rows
    print("\n4. Duplicate Rows:")
    num_duplicates = df.duplicated().sum()
    if num_duplicates == 0:
        print("No duplicate rows found.")
    else:
        print(f"Number of duplicate rows: {num_duplicates}")
    
    # 5. Outliers
    print("\n5. Outliers:")
    numerical_cols = df.select_dtypes(include=np.number).columns
    for col in numerical_cols:
        plt.figure(figsize=(8, 4))
        sns.boxplot(x=df[col])
        plt.title(f"Boxplot of {col}")
        plt.show()
    
    # 6. Data Distribution
    print("\n6. Data Distribution:")
    categorical_cols = df.select_dtypes(include='object').columns
    for col in categorical_cols:
        plt.figure(figsize=(8, 4))
        df[col].value_counts().plot(kind='bar')
        plt.title(f"Distribution of {col}")
        plt.show()
    
    # 7. Data Quality Metrics (Additional)
    print("\n7. Additional Data Quality Metrics:")
    # Calculate additional data quality metrics as needed
    
    # 8. Feature Engineering (Additional)
    print("\n8. Feature Engineering:")
    # Review newly engineered features as needed
    
    # 9. Data Imbalance (Additional)
    print("\n9. Data Imbalance:")
    # Check for class imbalances in classification tasks as needed
    
    # 10. Data Schema and Metadata (Additional)
    print("\n10. Data Schema and Metadata:")
    # Inspect data schema and metadata as needed
    
    # 11. Documentation and Annotations (Additional)
    print("\n11. Documentation and Annotations:")
    # Review documentation and annotations as needed

### Working With DataTypes

In [11]:
def inspect_and_optimize_data_types(df):
    """
    Inspect the data and convert data types based on user inputs for maximum memory efficiency.

    Parameters:
        df (DataFrame): The input DataFrame to be inspected and optimized.

    Returns:
        DataFrame: The DataFrame with optimized data types.

    Example usage:
        # Load data into a DataFrame (replace 'data.csv' with your data file)
        df = pd.read_csv('data.csv')

        # Inspect and optimize data types based on user inputs
        optimized_df = inspect_and_optimize_data_types(df)
    """
    # Get memory usage before optimization
    memory_before = df.memory_usage(deep=True).sum() / (1024**2)  # Convert bytes to megabytes

    # Inspect the data types of columns
    print("Initial Data Types:")
    print(df.dtypes)

    # User input to select data types to convert
    print("\nSelect data types to convert:")
    print("1. Integer")
    print("2. Floating-point")
    print("3. Object (String)")
    print("4. Categorical (String with limited unique values)")
    selected_types = input("Enter the numbers separated by commas (e.g., '1,2,4'): ").split(',')
    selected_types = [int(i) for i in selected_types]

    # User input to select columns for conversion
    print("\nSelect columns to convert (or enter 'all' to convert all columns):")
    print(df.columns)
    columns_to_convert = input("Enter column names separated by commas (or 'all'): ").split(',')
    columns_to_convert = [col.strip() for col in columns_to_convert]

    # Convert data types based on user inputs
    for data_type in selected_types:
        if data_type == 1:  # Integer
            if columns_to_convert[0] == 'all':
                int_cols = df.select_dtypes(include=['int']).columns
            else:
                int_cols = [col for col in columns_to_convert if col in df.columns]
            for col in int_cols:
                df[col] = pd.to_numeric(df[col], downcast='integer')
        elif data_type == 2:  # Floating-point
            if columns_to_convert[0] == 'all':
                float_cols = df.select_dtypes(include=['float']).columns
            else:
                float_cols = [col for col in columns_to_convert if col in df.columns]
            for col in float_cols:
                df[col] = pd.to_numeric(df[col], downcast='float')
        elif data_type == 3:  # Object (String)
            if columns_to_convert[0] == 'all':
                object_cols = df.select_dtypes(include=['object']).columns
            else:
                object_cols = [col for col in columns_to_convert if col in df.columns]
            for col in object_cols:
                df[col] = df[col].astype('category')
        elif data_type == 4:  # Categorical (String with limited unique values)
            if columns_to_convert[0] == 'all':
                object_cols = df.select_dtypes(include=['object']).columns
            else:
                object_cols = [col for col in columns_to_convert if col in df.columns]
            for col in object_cols:
                num_unique_values = len(df[col].unique())
                num_total_values = len(df[col])
                if num_unique_values / num_total_values < 0.5:  # Adjust the threshold as needed
                    df[col] = df[col].astype('category')

    # Get memory usage after optimization
    memory_after = df.memory_usage(deep=True).sum() / (1024**2)  # Convert bytes to megabytes

    print("\nData Types After Optimization:")
    print(df.dtypes)

    print("\nMemory Usage Before Optimization: {:.2f} MB".format(memory_before))
    print("Memory Usage After Optimization: {:.2f} MB".format(memory_after))
    print("Memory Usage Reduction: {:.2f}%".format((1 - memory_after / memory_before) * 100))

    return df

### Outlier Detection
1. z-score
2. percentile
3. IQR

Before Outlier Detection

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   age             32561 non-null  int32   
 1   workclass       32561 non-null  category
 2   fnlwgt          32561 non-null  int32   
 3   education       32561 non-null  category
 4   education.num   32561 non-null  int32   
 5   marital.status  32561 non-null  category
 6   occupation      32561 non-null  category
 7   relationship    32561 non-null  category
 8   race            32561 non-null  category
 9   sex             32561 non-null  category
 10  capital.gain    32561 non-null  int32   
 11  capital.loss    32561 non-null  int32   
 12  hours.per.week  32561 non-null  int32   
 13  native.country  32561 non-null  category
 14  income          32561 non-null  category
dtypes: category(9), int32(6)
memory usage: 1.0 MB


In [ ]:
cat_col

Index(['workclass', 'education', 'marital.status', 'occupation',
       'relationship', 'race', 'sex', 'native.country', 'income'],
      dtype='object')

#### Z-Score on Numeric Columns

In [ ]:
import scipy.stats as stats
from copy import deepcopy

dfz = deepcopy(df) #make a deep copy (for Z-score)

for col in num_col:
    dfz['stats.zscore'] = stats.zscore(dfz[col])
    threshold = 3
    dfz['outliers'] = np.where((dfz['stats.zscore'] - threshold > 0), True, np.where(dfz['stats.zscore'] + threshold < 0, True, False))
    dfz.drop((dfz[dfz['outliers'] == True]).index, inplace=True)
    dfz.drop(['outliers', 'stats.zscore'], axis=1, inplace=True)

% Data Remains - After Outlier

In [ ]:
print(round((dfz.shape[0]/32561)*100, 2),'%') #Pass in dataframe and use total rows

91.61 %


> Approx 9% Dropped, If Threshold = 2, We Lose 25%

#### Percentile on Numeric Columns - YOU SELECT LIMIT

In [ ]:
from copy import deepcopy

dfq = deepcopy(df) #make a deep copy (for quartiles)

for col in num_col:
    upper_limit = dfq[col].quantile(0.98) #select upperlimit
    lower_limit = dfq[col].quantile(0.02) #select lower limit
    dfq[col] = np.where(dfq[col] < lower_limit, lower_limit, dfq[col])
    dfq[col] = np.where(dfq[col] > upper_limit, upper_limit, dfq[col])


% Data Remains - After Outlier

In [ ]:
print(round((dfq.shape[0]/32561)*100, 2),'%') #Pass in dataframe and use total rows

100.0 %


> Didn't Change!, Theoretically Would Choose Outliers That Would Cause Problems

### Dataframe by Dtype

In [ ]:
num_col = df.select_dtypes(include=np.number).columns # selects all the numerical columns
cat_col = df.select_dtypes(include='category').columns # Selects all the categorical columns

### Label Encoder

#### Understand Options in Categories - Determine if Ordinal, One-Hot, Label

In [ ]:
df.select_dtypes(include='category').nunique()

workclass          9
education         16
marital.status     7
occupation        15
relationship       6
race               5
sex                2
native.country    42
income             2
dtype: int64

> Don't use one-hot/dummies or else you'll have large amount of columns (41 for native.country, 15 and 14 added for education and occupation, respectively)

In [ ]:
from sklearn.preprocessing import LabelEncoder

def label_encoder(data): #function created, especially since we are applying to multiple dataframes that we want to pass in
    """It Label encodes every categorical column ie. converts categories into numbers and returns the modified dataframe"""
    
    for col in data.select_dtypes(include='category').columns:
        #Instantiate
        encoder = LabelEncoder()
        #Fit + Transform
        data[col] = encoder.fit_transform(data[col])

    return data

# Pass in our dataframes
df = label_encoder(df) #regular
dfz = label_encoder(dfz) #z-score treated
dfq = label_encoder(dfq) #quantile treated

#### Confirm Label Encoding

In [ ]:
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,0,77053,11,9,6,0,1,4,0,0,4356,40,39,0
1,82,4,132870,11,9,6,4,1,4,0,0,4356,18,39,0
2,66,0,186061,15,10,6,0,4,2,0,0,4356,40,39,0
3,54,4,140359,5,4,0,7,4,4,0,0,3900,40,39,0
4,41,4,264663,15,10,5,10,3,4,0,0,3900,40,39,0


### Encode the Categrical Variables
- Seasons, Holiday, Functioning Day

In [ ]:
df.info() #Look for your categorical

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Date                       8760 non-null   datetime64[ns]
 1   Rented Bike Count          8760 non-null   int32         
 2   Hour                       8760 non-null   int32         
 3   Temperature(°C)            8760 non-null   float32       
 4   Humidity(%)                8760 non-null   int32         
 5   Wind speed (m/s)           8760 non-null   float32       
 6   Visibility (10m)           8760 non-null   int32         
 7   Dew point temperature(°C)  8760 non-null   float32       
 8   Solar Radiation (MJ/m2)    8760 non-null   float32       
 9   Rainfall(mm)               8760 non-null   float32       
 10  Snowfall (cm)              8760 non-null   float32       
 11  Seasons                    8760 non-null   category      
 12  Holida

In [ ]:
display(df['Holiday'].unique())
display(df['Functioning Day'].unique())
display(df['Seasons'].unique())

['No Holiday', 'Holiday']
Categories (2, object): ['Holiday', 'No Holiday']

['Yes', 'No']
Categories (2, object): ['No', 'Yes']

['Winter', 'Spring', 'Summer', 'Autumn']
Categories (4, object): ['Autumn', 'Spring', 'Summer', 'Winter']

In [ ]:
display(df['Holiday'].value_counts())
display(df['Functioning Day'].value_counts())
display(df['Seasons'].value_counts())

Holiday
No Holiday    8328
Holiday        432
Name: count, dtype: int64

Functioning Day
Yes    8465
No      295
Name: count, dtype: int64

Seasons
Spring    2208
Summer    2208
Autumn    2184
Winter    2160
Name: count, dtype: int64

In [ ]:
display(df['Holiday'].cat.codes.value_counts())
display(df['Functioning Day'].cat.codes.value_counts())
display(df['Seasons'].cat.codes.value_counts())

1    8328
0     432
Name: count, dtype: int64

1    8465
0     295
Name: count, dtype: int64

1    2208
2    2208
0    2184
3    2160
Name: count, dtype: int64

We want it to be Reversed for Holiday! 

### Get Dummies

In [ ]:
df['Holiday'] = df['Holiday'].apply(lambda x: 0 if x == 'No Holiday' else 1)
df['Functioning Day'] = df['Functioning Day'].apply(lambda x: 1 if x == 'Yes' else 0)
dummies = pd.get_dummies(df['Seasons'], drop_first=True) #drop_first drops first category to avoid multicollinearity (drops 1 of the seasons as it will be implicitly represented)
dummies = dummies.astype(int) #in case get_dummies gives you boolean (otherwise it should default)
df = pd.concat([df.drop('Seasons', axis=1), dummies], axis=1) #remove original 'Seasons' column, merge dummy variables with remaining DataFrame along the columns axis 1

In [ ]:
df = pd.get_dummies(df, columns=['ChestPain', 'Thal'], dtype=int)

CONFIRM THAT IT WORKED

In [ ]:
display(df)
display(df.info())

,Date,Rented Bike Count,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Holiday,Functioning Day,Spring,Summer,Winter
0,2017-12-01,254,0,-5.2,37,2.2,2000,-17.600000,0.0,0.0,0.0,0,1,0,0,1
1,2017-12-01,204,1,-5.5,38,0.8,2000,-17.600000,0.0,0.0,0.0,0,1,0,0,1
2,2017-12-01,173,2,-6.0,39,1.0,2000,-17.700001,0.0,0.0,0.0,0,1,0,0,1
3,2017-12-01,107,3,-6.2,40,0.9,2000,-17.600000,0.0,0.0,0.0,0,1,0,0,1
4,2017-12-01,78,4,-6.0,36,2.3,2000,-18.600000,0.0,0.0,0.0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2018-11-30,1003,19,4.2,34,2.6,1894,-10.300000,0.0,0.0,0.0,0,1,0,0,0
8756,2018-11-30,764,20,3.4,37,2.3,2000,-9.900000,0.0,0.0,0.0,0,1,0,0,0
8757,2018-11-30,694,21,2.6,39,0.3,1968,-9.900000,0.0,0.0,0.0,0,1,0,0,0
8758,2018-11-30,712,22,2.1,41,1.0,1859,-9.800000,0.0,0.0,0.0,0,1,0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Date                       8760 non-null   datetime64[ns]
 1   Rented Bike Count          8760 non-null   int32         
 2   Hour                       8760 non-null   int32         
 3   Temperature(°C)            8760 non-null   float32       
 4   Humidity(%)                8760 non-null   int32         
 5   Wind speed (m/s)           8760 non-null   float32       
 6   Visibility (10m)           8760 non-null   int32         
 7   Dew point temperature(°C)  8760 non-null   float32       
 8   Solar Radiation (MJ/m2)    8760 non-null   float32       
 9   Rainfall(mm)               8760 non-null   float32       
 10  Snowfall (cm)              8760 non-null   float32       
 11  Holiday                    8760 non-null   category      
 12  Functi

None

### Further Inspecting
* Unique Values and Number of Unique Values
* Missing
* Outliers

# 3. Data Quality Metrics

In [58]:
import pprint

def calculate_data_quality_metrics(df):
    """
    Calculate additional data quality metrics for the dataset.

    Parameters:
        df (DataFrame): The input DataFrame.

    Returns:
        dict: A dictionary containing data quality metrics.

    Example usage:
        # Load data into a DataFrame (replace 'data.csv' with your data file)
        df = pd.read_csv('data.csv')

        # Calculate additional data quality metrics
        quality_metrics = calculate_data_quality_metrics(df)
        print("Additional Data Quality Metrics:")
        print(quality_metrics)
    """
    metrics = {}

    # Completeness: Percentage of non-null values for each column
    completeness = df.notnull().mean() * 100
    metrics['completeness'] = completeness

    # Accuracy: For categorical columns, percentage of unique values that are correct
    accuracy = {}
    categorical_cols = df.select_dtypes(include='object').columns
    for col in categorical_cols:
        accuracy[col] = df[col].nunique() / df[col].notnull().sum() * 100
    metrics['accuracy'] = accuracy

    # Consistency: Percentage of unique values that are consistent across different columns
    consistency = {}
    for col in df.columns:
        if df[col].dtype == 'object':  # Consider only categorical columns
            unique_values = df[col].unique()
            consistent_values = sum(df.apply(lambda x: x[col] in unique_values, axis=1)) / len(df) * 100
            consistency[col] = consistent_values
    metrics['consistency'] = consistency

    # Timeliness: Percentage of recent data points compared to the total number of data points
    # (This metric may not be applicable to all datasets)
    # Example: if there's a date column, you can calculate the percentage of data points within the last year
    timeliness = None  # Add implementation as needed
    metrics['timeliness'] = timeliness

    return metrics

In [75]:
pp = pprint.PrettyPrinter(indent=4)
quality_metrics = calculate_data_quality_metrics(df)
print("Additional Data Quality Metrics:")
pp.pprint(quality_metrics)

Additional Data Quality Metrics:
{   'accuracy': {   'Cabin': 72.05882352941177,
                    'Embarked': 0.3374578177727784,
                    'Name': 100.0,
                    'Sex': 0.22446689113355783,
                    'Ticket': 76.43097643097643},
    'completeness': PassengerId    100.000000
Survived       100.000000
Pclass         100.000000
Name           100.000000
Sex            100.000000
Age             80.134680
SibSp          100.000000
Parch          100.000000
Ticket         100.000000
Fare           100.000000
Cabin           22.895623
Embarked        99.775533
dtype: float64,
    'consistency': {   'Cabin': 22.895622895622896,
                       'Embarked': 99.77553310886644,
                       'Name': 100.0,
                       'Sex': 100.0,
                       'Ticket': 100.0},
    'timeliness': None}


: 

# 4. Data Cleaning & Preparation
* Need to Add
    * Handling Missing in Multiple Ways: dropna, fillna
    * Different Data Formating

### Fix DType: Datetime, Categorical, Float/Int

In [ ]:
def fix_dtype():
    """This is a static function that reduces memory consuption by converting objects 
    to category and int64 -> int8, float64 -> float16
    
    Returns an optimized DF"""
    print('Printing out categorical variables and the values \n\n')
    for col in df.columns:
        if col == 'Date':
            df[col] = pd.to_datetime(df[col],format='%d/%m/%Y')
        elif df[col].dtype == 'O':
            print("    ", col, df[col].unique())
            df[col] = pd.Categorical(df[col])
        elif df[col].dtype == 'float64':
             df[col] = df[col].astype('float32') #interestingly if used 'float16' it would return "inf" in your df.describe().T in T mean (so changed to float32)
        elif df[col].dtype == 'int64':
             df[col] = df[col].astype('int32')       
        
fix_dtype()

### Duplicates, Missing, Standardized String Formats

In [24]:
def clean_data(df, tasks):
    """
    Perform selected data cleaning tasks on a Pandas DataFrame.

    Parameters:
        df (DataFrame): The input DataFrame to be cleaned.
        tasks (list): A list of strings representing the cleaning tasks to be performed.

    Returns:
        DataFrame: The cleaned DataFrame.
    """
    cleaned_df = df.copy()

    for task in tasks:
        if task == 'remove_duplicates':
            cleaned_df = remove_duplicates(cleaned_df)
        elif task == 'handle_missing_values':
            cleaned_df = handle_missing_values(cleaned_df)
        elif task == 'standardize_data_formats':
            cleaned_df = standardize_data_formats(cleaned_df)
        else:
            print(f"Warning: Unknown task '{task}'")

    return cleaned_df

def remove_duplicates(df):
    """
    Remove duplicate rows from a DataFrame.

    Parameters:
        df (DataFrame): The input DataFrame.

    Returns:
        DataFrame: The DataFrame with duplicate rows removed.
    """
    return df.drop_duplicates()

def handle_missing_values(df):
    """
    Handle missing values in a DataFrame.

    Parameters:
        df (DataFrame): The input DataFrame.

    Returns:
        DataFrame: The DataFrame with missing values handled.
    """
    return df.dropna()

def standardize_data_formats(df):
    """
    Standardize data formats in a DataFrame.

    Parameters:
        df (DataFrame): The input DataFrame.

    Returns:
        DataFrame: The DataFrame with standardized data formats.
    """
    string_columns = df.select_dtypes(include='object').columns
    df[string_columns] = df[string_columns].apply(lambda x: x.str.lower())
    return df

In [ ]:
# Ask user for cleaning tasks
selected_tasks = []
print("Available cleaning tasks:")
print("1. Remove duplicates")
print("2. Handle missing values")
print("3. Standardize data formats")
while True:
    task = input("Enter the number of the cleaning task you want to perform (or 'done' to finish): ")
    if task.lower() == 'done':
        break
    elif task == '1':
        selected_tasks.append('remove_duplicates')
    elif task == '2':
        selected_tasks.append('handle_missing_values')
    elif task == '3':
        selected_tasks.append('standardize_data_formats')
    else:
        print("Invalid input. Please enter a valid task number.")

# Clean the data based on selected tasks
cleaned_df = clean_data(df, selected_tasks)
print("Data cleaning completed.")

# Inspect After Cleaning